## A macro to write the sizes of arrays in an expression

Let's first define a function with a lot of arrays, let's construct an example

In [ ]:
include("./DimensionChecker.jl"); using .DimensionChecker; # a dot is needed because DimensionChecker is a local module

In [2]:
x = ones(3,3);
z = ones(3,3);
y = ones(5,3);
u = ones(3,3);
v = ones(3,3);

ex = :(((x + z * y)*u^2)*v)

:(((x + z * y) * u ^ 2) * v)

In [3]:
@dimensionCheck ((x + z * y)*u^2)*v

Creating stack
Creating symbol size array


"Error in expression 'z * y' with dimensions '(3, 3) * (5, 3)'"

## Trying out on 2D linear convection

In [ ]:
nx = 81
ny = 81
nt = 100
c = 1
Δx = 2 / (nx - 1)
Δy = 2 / (ny - 1)
Δt = 1.
u = ones(ny,nx);      ##create a 1xn vector of 1's
u[convert(Int8,floor(0.5/Δy)):convert(Int8,floor(1/Δy+1)),
    convert(Int8,floor(0.5/Δx)):convert(Int8,floor(1/Δy+1))] .= 2;  #then set u = 2 between 0.5 and 1 as per our I.C.s


In [ ]:
un = deepcopy(u)
ex = :(u[2:end, 2:end] = (un[2:end, 3:end] - (c * Δt / Δx * 
        (un[2:end, 2:end] - un[2:end, 1:end-1])) - 
              (c * Δt / Δy * 
            (un[2:end, 2:end] - un[1:end-1, 2:end]))))

In [ ]:
(a,b) = init_traverse(ex)
expr_to_size_string(a,b)

In [1]:
import TreeView

function init_traverse(ex::Expr)
    (exp_stack,symbol_size) = traverse(ex);
    return (pop!(exp_stack),symbol_size)
end

function traverse(ex::Expr)
    exp_stack = [];
    symbol_size = Dict();
    traverse(ex,exp_stack,symbol_size)
end

function traverse(ex::Symbol,exp_stack,symbol_size)
    try symbol_size[string(ex)] = size(eval(ex))
        catch e  #not the nicest code  
    end
end

function traverse(ex::Expr,exp_stack,symbol_size) 
    if ex.head == :call  # function call 
        arg = ex.args[1];
        if (string(arg) != ":")
            test!(arg,exp_stack,symbol_size)
        end
        # Here all the operators are skipped because they are the first argument
        for arg in ex.args[2:end]
            if (string(arg) != ":")
                test!(arg,exp_stack,symbol_size)
            end
            traverse(arg,exp_stack,symbol_size);  # recursive
        end
    
        elseif ex.head == :ref # now where at the indexing level and stop here
        symbol_size[string(ex)] = size(eval(ex));
        a = size(eval(ex));
        
    else
        for arg in ex.args
            test!(arg,exp_stack,symbol_size)
            traverse(arg,exp_stack,symbol_size);  # recursive
        end
    end

    
    return (exp_stack,symbol_size)
end

function traverse(ex,exp_stack,symbol_size)
end

function test!(ex::Expr,exp_stack,symbol_size)
    try 
        eval(ex);
    catch e
        push!(exp_stack,ex);
    end
end

function test!(ex,exp_stack,symbol_size)
    # If not an expression just ignore
end

function expr_to_size_string(ex::Expr,symbol_size::Dict)
    str = string(ex);
    size_str = str
    for (key, value) in symbol_size
        size_str = replace(size_str, string(key) => string(value))
    end
    return "Error in expression '" * str * "' with dimensions '" * size_str *"'"
end

macro dimensionCheck(ex)
    (exp_stack,symbol_size) = init_traverse(esc(ex))
    return expr_to_size_string(exp_stack,symbol_size)
end

@dimensionCheck (macro with 1 method)

## Failed attempts

In [ ]:
function replace_array(arg)
    println("$(string(eval(arg)))")
    println("$(isa(eval(arg),Array))")
    arg = (size(eval(arg)))
    return arg
end
    
function traverse_replace(ex)
    return ex
end

function traverse_replace(ex::Expr)
    for (index,arg) in enumerate(ex.args)
        if isa(eval(arg),Array)
            arg = replace_array(arg)
            arg = traverse_replace(arg)
        else
            arg = traverse_replace(arg)  # recursive
        end
        ex.args[index]=arg
    end
    return(ex)
end

function check_size(ex::Expr)
    return string(traverse_replace(ex))
end

macro size_check(ex)
    answer = :(check_size(esc(ex)))
    return answer
end

In [ ]:
using TreeView
@tree ((((x + z * y)*u^2)*v))

In [ ]:
x = ones(3,3);
y = ones(2,3);
z = ones(3,2);
u = ones(3,3);
v = ones(3,11);

ex = :((((x + z * y)*u^2)*v))

check_size(ex)

In [ ]:
x = ones(3,3);
y = ones(2,3);
z = ones(3,2);
u = ones(3,3);
v = ones(3,10);

@size_check ((((x + z * y)*u^2)*v))